In [ ]:
import xarray as xr
import pyfesom2 as pf
import numpy as np
from matplotlib import pyplot as plt
from joblib import Parallel, delayed
import datetime as dt
import cftime as cf
from scipy import stats as st
from matplotlib.gridspec import GridSpec
from scipy import signal

In [ ]:
timed=xr.open_dataset('c6_ssh-eke_mon_lin_eq_ann_node_r'+str(1)+'_1860-2090_nol_21yr_anom_rename_weighted.nc')
obstime=xr.open_dataset('obs_eke_1993-2020_nol_5d_rm_runmean_mdata_weighted.nc')
temptime=xr.open_dataset('c6_tas_r'+str(1)+'_1850-2100_ymonmean_aweight_fldmean_anom.nc')
timed.time

In [ ]:
#load the saved time series
kuro1=np.load('annual_kuro_basin_ts_r1.npy')
kuro2=np.load('annual_kuro_basin_ts_r2.npy')
kuro3=np.load('annual_kuro_basin_ts_r3.npy')
kuro4=np.load('annual_kuro_basin_ts_r4.npy')
kuro5=np.load('annual_kuro_basin_ts_r5.npy')

gulf1=np.load('annual_gulf_basin_ts_r1.npy')
gulf2=np.load('annual_gulf_basin_ts_r2.npy')
gulf3=np.load('annual_gulf_basin_ts_r3.npy')
gulf4=np.load('annual_gulf_basin_ts_r4.npy')
gulf5=np.load('annual_gulf_basin_ts_r5.npy')

agul1=np.load('annual_agul_basin_ts_r1.npy')
agul2=np.load('annual_agul_basin_ts_r2.npy')
agul3=np.load('annual_agul_basin_ts_r3.npy')
agul4=np.load('annual_agul_basin_ts_r4.npy')
agul5=np.load('annual_agul_basin_ts_r5.npy')

braz1=np.load('annual_braz_basin_ts_r1.npy')
braz2=np.load('annual_braz_basin_ts_r2.npy')
braz3=np.load('annual_braz_basin_ts_r3.npy')
braz4=np.load('annual_braz_basin_ts_r4.npy')
braz5=np.load('annual_braz_basin_ts_r5.npy')

accu1=np.load('annual_accu_basin_ts_r1.npy')
accu2=np.load('annual_accu_basin_ts_r2.npy')
accu3=np.load('annual_accu_basin_ts_r3.npy')
accu4=np.load('annual_accu_basin_ts_r4.npy')
accu5=np.load('annual_accu_basin_ts_r5.npy')

In [ ]:
#load the mgst time series
mgst1=xr.open_dataset('c6_tas_r1_1850-2100_ymonmean_aweight_fldmean_anom.nc').tas[:,0,0]
mgst2=xr.open_dataset('c6_tas_r2_1850-2100_ymonmean_aweight_fldmean_anom.nc').tas[:,0,0]
mgst3=xr.open_dataset('c6_tas_r3_1850-2100_ymonmean_aweight_fldmean_anom.nc').tas[:,0,0]
mgst4=xr.open_dataset('c6_tas_r4_1850-2100_ymonmean_aweight_fldmean_anom.nc').tas[:,0,0]
mgst5=xr.open_dataset('c6_tas_r5_1850-2100_ymonmean_aweight_fldmean_anom.nc').tas[:,0,0]

In [ ]:
def stand(ts):
    hist=ts[133:133+28]
    hm=np.mean(hist)
    hstd=np.std(signal.detrend(hist))
    ts=(ts-hm)/hstd
    return ts

In [ ]:
kuros=[kuro1,kuro2,kuro3,kuro4,kuro5]
gulfs=[gulf1,gulf2,gulf3,gulf4,gulf5]
brazs=[braz1,braz2,braz3,braz4,braz5]
aguls=[agul1,agul2,agul3,agul4,agul5]
accus=[accu1,accu2,accu3,accu4,accu5]
mgsts=[mgst1,mgst2,mgst3,mgst4,mgst5]

In [ ]:
kurox=[]
gulfx=[]
brazx=[]
agulx=[]
accux=[]
for i in np.arange(len(kuros)):
    kurox.append(stand(kuros[i]))
    gulfx.append(stand(gulfs[i]))
    brazx.append(stand(brazs[i]))
    accux.append(stand(accus[i]))
    agulx.append(stand(aguls[i]))

In [ ]:
#create one figure

In [ ]:
labels=['a. Gulf Stream','b. Kuroshio','c. Agulhas','d. Brazil/Malvinas','e. Antarctic Circumpolar Current']
basins=[gulfx,kurox,agulx,brazx,accux]
temps=[mgst1,mgst2,mgst3,mgst4,mgst5]

In [ ]:
fig=plt.figure(figsize=(24,16),dpi=700)
fig,ax=plt.subplots(3,2,figsize=(24,16))

In [ ]:
fig=plt.figure(figsize=(24,16),dpi=800)

lax = fig.add_subplot(111)
lax.tick_params(labelcolor='w',top=False,bottom=False,left=False,right=False)
lax.spines['top'].set_color('none')
lax.spines['bottom'].set_color('none')
lax.spines['left'].set_color('none')
lax.spines['right'].set_color('none')
lax.set_ylabel('Eddy kinetic energy ('+r'$\sigma$'+' from mean)',labelpad=40,fontsize=32)
lax.set_xlabel('Mean annual surface temperature anomaly (°C)',labelpad=14,fontsize=28)

ax=np.hstack(ax)
gs=GridSpec(3,2,figure=fig)

for i in np.arange(5):
    if i ==4:
        ax[i]=fig.add_subplot(gs[2,0:])
        ax[i].set_yticks([-3,0,3,6,9])
    else:
        ax[i]=fig.add_subplot(3,2,i+1)
        
    mgstf=np.hstack([mgst1[154:-10],mgst2[154:-10],mgst3[154:-10],mgst4[154:-10],mgst5[154:-10]])
    af,bf,rf,pf,stf=st.linregress(mgstf,np.hstack([basins[i][0][144:],basins[i][1][144:],basins[i][2][144:],basins[i][3][144:],basins[i][4][144:]]))
    trf=np.polyval([af,bf],np.linspace(np.min(mgstf),np.max(mgstf)))
    
    if pf<0.001:
        text='{:.2f}'.format(np.round(af,2))+' '+r'$\sigma$'+'/°C, R$^2$ = '+'{:.2f}'.format(np.round(rf**2,2))+', p<0.001'
    else:
        text='{:.2f}'.format(np.round(af,2))+' '+r'$\sigma$'+'/°C, R$^2$ = ''{:.2f}'.format(np.round(rf**2,2))+', p = '+'{:.3f}'.format(np.round(pf,3))

    if i==0:
        ll='upper right'
    else:
        ll='lower right'
    
    plt.plot(np.linspace(np.min(mgstf),np.max(mgstf)),trf,color='red',ls='--',lw=2,
            label=text)
    ax[i].legend(loc=ll,fontsize=20,framealpha=0.5)
    
    for j in np.arange(5):
        ax[i].scatter(mgsts[j][10:-10],basins[i][j],color='grey',alpha=0.8,lw=0.4,)
        
    ax[i].tick_params(labelsize=18)
    ax[i].text(0.02, 0.92,labels[i],horizontalalignment='left', verticalalignment='center', 
                   transform=ax[i].transAxes,fontsize=25,weight='bold')
    ax[i].grid(which='major',axis='y')

fig.tight_layout(h_pad=1.5,w_pad=0.5)
fig.savefig('Figure_4.png', bbox_inches="tight")